Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)


In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [106]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
# Subset the data by BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and SALE_PRICE > $100,000 & < $2,000,000
df_ntm = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'].copy()
df_sub = df_ntm[(df_ntm['SALE_PRICE'] >= 100000) & (df_ntm['SALE_PRICE'] <= 2000000)].copy()


In [0]:
# Standardize the date value
df_sub['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff_a = pd.to_datetime('2019-04-01')
cutoff_b = pd.to_datetime('2019-03-31')

# Convert features to string types
df_sub['TAX_CLASS_AT_PRESENT'] = df_sub['TAX_CLASS_AT_PRESENT'].apply(str)
df_sub['TAX_CLASS_AT_TIME_OF_SALE'] = df_sub['TAX_CLASS_AT_TIME_OF_SALE'].apply(str)

# Construct the train dataset
df_train = df_sub[df_sub['SALE_DATE'] < cutoff_a].copy()
df_test  = df_sub[df_sub['SALE_DATE'] > cutoff_b].copy()


In [109]:
df_train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2517,1,01 ONE FAMILY DWELLINGS,2517,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_TIME_OF_SALE,2517,1,1,2517,NaT,NaT
TAX_CLASS_AT_PRESENT,2517,2,1,2486,NaT,NaT
BOROUGH,2517,5,4,1209,NaT,NaT
NEIGHBORHOOD,2517,6,OTHER,2390,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921,NaT,NaT
SALE_DATE,2517,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
LAND_SQUARE_FEET,2517,889,"4,000",235,NaT,NaT


In [0]:
# Define target and high cardinality features
target = 'SALE_PRICE'
high_cardinality = ['SALE_DATE', 'LAND_SQUARE_FEET', 'ADDRESS']
empty_feature = 'EASE-MENT'

# Drop the non modeling features
features = df_train.columns.drop([target] + high_cardinality + [empty_feature])

df_train_X = df_train[features]
df_train_y = df_train[target]

df_test_X = df_test[features]
df_test_y = df_test[target]

In [0]:
# Conduct OneHotEncoding on the training set
import category_encoders as ce
encdr = ce.OneHotEncoder(use_cat_names=True)

# Fit the encoder (determine the categories)
encdr.fit(df_train_X)

# Transform the category data to numeric values
df_train_X = encdr.transform(df_train_X)
df_test_X  = encdr.transform(df_test_X)


In [0]:
# Select the best 15 features that correlate the best with the target
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=15)

In [113]:
# Use the training data set to select the best correlated features
df_train_X_selctd = selector.fit_transform(df_train_X, df_train_y)
df_test_X_selctd  = selector.transform(df_test_X)
df_train_X_selctd.shape, df_test_X_selctd.shape

((2517, 15), (647, 15))

In [114]:
# List out the selected columns
df_train_cols = df_train_X.columns
col_sel       = selector.get_support()
df_train_cols_selected = df_train_cols[col_sel]

# Print out selected columns
lst_col = list(df_train_cols_selected)
lst_col.sort()

print(f'Selected Columns:\n')
for col in lst_col:
  print(col)

Selected Columns:

APARTMENT_NUMBER_RP.
BLOCK
BOROUGH_1
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_B3
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_S0
GROSS_SQUARE_FEET
LOT
NEIGHBORHOOD_ASTORIA
NEIGHBORHOOD_FOREST HILLS
ZIP_CODE


In [0]:
# Conduct Ridge Regression over a range of alpha parameters
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

lst_alpha, lst_mae, lst_coef, lst_intcpt, lst_cols = [], [], [], [], []

# Iterate over alpha values
for alpha in [10**0, 10**1, 10**2, 10**3, 10**4, 10**5]:
  # Scale the feature data
  scalr = StandardScaler()
  df_train_X_scaled = scalr.fit_transform(df_train_X)
  df_test_X_scaled  = scalr.transform(df_test_X)

  # Fit the Ridge regression model 
  mdl = Ridge(alpha=alpha)
  mdl.fit(df_train_X_scaled, df_train_y)
  y_pred = mdl.predict(df_test_X_scaled)

  # Determine the mean absolute error
  mae = mean_absolute_error(df_test_y, y_pred)

  # Capture model execution elements
  lst_alpha.append(alpha)
  lst_mae.append(mae)
  lst_coef.append(mdl.coef_)
  lst_intcpt.append(mdl.intercept_)
  lst_cols.append(df_train_X.columns)


In [129]:
# Determine the smallest mae
idx = lst_mae.index(min(lst_mae))

# Construct equation "terms"
lst_features = []
for str in lst_cols[idx]:
  lst_features.append("*" + str)

# Create a dataframe containing regression equation terms
df_output = pd.DataFrame({'Value': lst_coef[idx], 'Feature': lst_features})
df_output.sort_values(by='Value', ascending=False, inplace=True)

# Add the intercept value to the equation dataframe
df_output = df_output.append(pd.DataFrame([[lst_intcpt[idx], "(intercept)"]], columns=df_output.columns))

print(f'Regression Expression with the Lowest MAE: {round(lst_mae[idx], 2)} leveraged alpha value: {lst_alpha[idx]}\n')
df_output

Regression Expression with the Lowest MAE: 161859.66 leveraged alpha value: 100



,Value,Feature
35,115260.791692,*GROSS_SQUARE_FEET
0,57705.148051,*BOROUGH_3
1,32765.639479,*BOROUGH_4
31,29803.278718,*ZIP_CODE
36,26757.059950,*YEAR_BUILT
8,22172.088563,*NEIGHBORHOOD_FOREST HILLS
9,10842.140642,*NEIGHBORHOOD_BOROUGH PARK
28,10531.447181,*BUILDING_CLASS_AT_PRESENT_B3
6,10048.180145,*NEIGHBORHOOD_FLUSHING-NORTH
21,9932.899286,*BUILDING_CLASS_AT_PRESENT_A3
